In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install cellpose==3.0.8

In [3]:
import os

DATASETS_BASE_PATH = "/content/drive/MyDrive/szakmai gyakorlat/cellpose-cell-counting/datasets"

OLD_FOLDER = "annotated"
CUSTOM_ANNOTATED_FOLDER = "2024.05.09. Ac-kPHSRN+fibronektin-few-annotated"

OLD_DATASET_PATH = os.path.join(DATASETS_BASE_PATH, OLD_FOLDER + ".zip")
CUSTOM_ANNOTATED_DATASET_PATH = os.path.join(DATASETS_BASE_PATH, CUSTOM_ANNOTATED_FOLDER + ".zip")

In [ ]:
!unzip "{OLD_DATASET_PATH}" -d "/content"
!unzip "{CUSTOM_ANNOTATED_DATASET_PATH}" -d "/content/{CUSTOM_ANNOTATED_FOLDER}"

In [5]:
import glob
import numpy as np
import cv2
from PIL import Image

training_images, training_masks = [], []

old_paths = glob.glob(OLD_FOLDER + "/*.jpg")
custom_annotated_paths = glob.glob(CUSTOM_ANNOTATED_FOLDER + "/*.jpg")

for path in old_paths:
  img = cv2.imread(path, 0) # grayscale
  mask = np.load(path.replace(".jpg", "_seg.npy"), allow_pickle=True).item()["masks"]
  training_images.append(img)
  training_masks.append(mask)

for path in custom_annotated_paths:
  img = cv2.imread(path, 0) # grayscale
  mask = np.array(Image.open(path.replace(".jpg", "_mask.tiff")))
  training_images.append(img)
  training_masks.append(mask)

In [6]:
from cellpose import models, train
from tqdm import trange
import torch
import logging

def train_model(model_type, n_epochs):
  logger = logging.getLogger('cellpose')
  logger.setLevel(logging.DEBUG)

  model = models.CellposeModel(model_type=model_type, gpu=torch.cuda.is_available())

  train.train_seg(
              model.net, n_epochs=n_epochs,
              train_data=training_images, train_labels=training_masks,
              channels=[0, 0], normalize=True,
              weight_decay=1e-4, SGD=True, learning_rate=0.1,
              save_path="/content/drive/MyDrive/szakmai gyakorlat/cellpose-cell-counting", model_name=f"{model_type}_all_annotated")

In [7]:
train_model("cyto3", n_epochs = 1000)

INFO:cellpose.models:>> cyto3 << model set to be used
INFO:cellpose.models:Downloading: "https://www.cellpose.org/models/cyto3" to /root/.cellpose/models/cyto3

100%|██████████| 25.3M/25.3M [00:03<00:00, 8.68MB/s]
INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>>>> loading model /root/.cellpose/models/cyto3
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
INFO:cellpose.dynamics:computing flows for labels
100%|██████████| 27/27 [00:12<00:00,  2.08it/s]
INFO:cellpose.train:>>> computing diameters
100%|██████████| 27/27 [00:00<00:00, 396.96it/s]
INFO:cellpose.train:>>> using channels [0, 0]
INFO:cellpose.train:>>> normalizing {'lowhigh': None, 'percentile': None, 'normalize': True, 'norm3D': False, 'sharpen_radius': 0, 'smooth_radius': 0, 'tile_norm_blocksize': 0, 'tile_norm_smooth3D': 1, 'invert': False}
INFO:cellpose.train:>>> n_epochs=1000, n_train=27, n_test=None
